In [1]:
%AddDeps org.bdgenomics.adam adam-core-spark2_2.11 0.24.0
%AddDeps comp.bio.aging adam-playground_2.11 0.0.13 --repository https://dl.bintray.com/comp-bio-aging/main/

Marking org.bdgenomics.adam:adam-core-spark2_2.11:0.24.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir13940453237730335/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir13940453237730335/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.jar
-> New file at /tmp/toree-tmp-dir13940453237730335/toree_add_deps/https/repo1.maven.org/maven2/org/bdgenomics/adam/adam-core-spark2_2.11/0.24.0/adam-core-spark2_2.11-0.24.0.pom


Waiting for a Spark session to start...

Marking comp.bio.aging:adam-playground_2.11:0.0.13 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir13940453237730335/toree_add_deps/
-> https://dl.bintray.com/comp-bio-aging/main/
-> https://repo1.maven.org/maven2


In [3]:
%AddDeps org.apache.hadoop hadoop-azure 2.7.6
%AddDeps com.microsoft.azure azure-storage 2.0.0

Marking org.apache.hadoop:hadoop-azure:2.7.6 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir13940453237730335/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir13940453237730335/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.jar
-> New file at /tmp/toree-tmp-dir13940453237730335/toree_add_deps/https/repo1.maven.org/maven2/org/apache/hadoop/hadoop-azure/2.7.6/hadoop-azure-2.7.6.pom


Waiting for a Spark session to start...

Marking com.microsoft.azure:azure-storage:2.0.0 for download
Preparing to fetch from:
-> file:/tmp/toree-tmp-dir13940453237730335/toree_add_deps/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree-tmp-dir13940453237730335/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.jar
-> New file at /tmp/toree-tmp-dir13940453237730335/toree_add_deps/https/repo1.maven.org/maven2/com/microsoft/azure/azure-storage/2.0.0/azure-storage-2.0.0.pom


In [2]:
import  org.apache.spark._
import org.apache.spark.sql.{DataFrame, Encoders, SparkSession}
import org.apache.spark.sql.types.StructType
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._

In [5]:
def sparkHadoopConf(sc: SparkContext, acountName: String, accountKey: String) = {
  sc.hadoopConfiguration.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
  sc.hadoopConfiguration.set("fs.azure.account.key." + acountName + ".blob.core.windows.net", accountKey)
  sc
}

sparkHadoopConf: (sc: org.apache.spark.SparkContext, acountName: String, accountKey: String)org.apache.spark.SparkContext


In [6]:
def azurize(container: String, accountName: String, blobFile: String): String = "wasbs://"+container+"@"+accountName+".blob.core.windows.net/"+blobFile 

def writeText2Azure[T]( rdd: RDD[T], container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  rdd.saveAsTextFile(url)
  url
}

def writeTsv2Azure( df: DataFrame, container: String, accountName: String, blobFile: String ): String =
{
  val url = azurize(container, accountName, blobFile)
  df.write.option("sep","\t").option("header","true").csv(url)
  url
}

azurize: (container: String, accountName: String, blobFile: String)String
writeText2Azure: [T](rdd: org.apache.spark.rdd.RDD[T], container: String, accountName: String, blobFile: String)String
writeTsv2Azure: (df: org.apache.spark.sql.DataFrame, container: String, accountName: String, blobFile: String)String


In [7]:
val account = "pipelines1"
val key = ""
val connString = s"DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=${key};EndpointSuffix=core.windows.net"
def az(path: String): String = azurize("storage", account, path)

account = pipelines1
key = 
connString = DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net


az: (path: String)String


DefaultEndpointsProtocol=https;AccountName=pipelines1;AccountKey=;EndpointSuffix=core.windows.net

In [8]:
val sparkContext = sc
sparkHadoopConf(sparkContext, account, key)
  
val spark = SparkSession
  .builder()
  .appName("mapping_models")
  .getOrCreate()

sparkContext = org.apache.spark.SparkContext@50ab59c2
spark = org.apache.spark.sql.SparkSession@542f5bed


Spark app-20180612163411-0003: Some(http://acfd6de92b88:4041)

In [9]:
import org.apache.spark.sql.functions._
import spark.implicits._

val toDouble = udf[Double, String]( _.toDouble)

toDouble = UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))


error: error while loading QualifiedTableName, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/QualifiedTableName.class)' has location not matching its contents: contains class QualifiedTableName
error: error while loading JavaTypeInference, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/JavaTypeInference.class)' has location not matching its contents: contains class JavaTypeInference
error: error while loading FunctionIdentifier, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/FunctionIdentifier.class)' has location not matching its contents: contains class FunctionIdentifier
error: error while loading DefinedByConstructorParams, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/DefinedByConstructorParams.class)' has location 

UserDefinedFunction(<function1>,DoubleType,Some(List(StringType)))

In [13]:
val gtexPath = "/GTEx"
val genesPath = az(gtexPath + "/gtex_genes.tsv")

//val txtPath = az(gtexPath + "/GTEx_Analysis_2016-01-15_v7_RSEMv1.2.22_transcript_tpm.txt")
//val gctPath = az(gtexPath + "/GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct")


Name: Compile Error
Message: <console>:100: error: not found: value expressionsPath
       val validationPath = expressionsPath + "/" + "validation"
                            ^

StackTrace: 

In [16]:
def write(df: DataFrame, url: String, header: Boolean = true) = {
  df.coalesce(1).write.option("sep","\t").option("header", header).csv(url)
  url
}

write: (df: org.apache.spark.sql.DataFrame, url: String, header: Boolean)String


In [12]:
def readTSV(path: String, header: Boolean = false, sep: String = "\t"): DataFrame = spark.read
    .option("sep", sep)
    .option("comment", "#")
    .option("inferSchema", true)
    .option("header", header)
    .option("ignoreLeadingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("ignoreTrailingWhiteSpace", true)
    .option("maxColumns", 150000)
    .csv(path)

readTSV: (path: String, header: Boolean, sep: String)org.apache.spark.sql.DataFrame


In [15]:
 def toVectors(dataFrame: DataFrame, columns: Seq[String], output: String) = {  
      import org.apache.spark.ml.feature.VectorAssembler
      import org.apache.spark.ml.linalg.Vectors

      val assembler = new VectorAssembler()
        .setInputCols(columns.toArray)
        .setOutputCol(output)

      assembler.transform(dataFrame.na.fill(0.0, columns).na.fill("")).select(output)
    }

toVectors: (dataFrame: org.apache.spark.sql.DataFrame, columns: Seq[String], output: String)org.apache.spark.sql.DataFrame


In [17]:
//for testing
val expressionsPath = "expressions"
val byGoPath = expressionsPath + "/go"
val comparison = byGoPath + "/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv"
val grouped = byGoPath + "/grouped"
val ranked = byGoPath + "/grouped/ranked"
val transcriptsPath = expressionsPath + "/transcripts"

val validationPath = expressionsPath + "/" + "validation"
val mouseValidationPath = validationPath + "/" + "mouse"
val jenageValidationPath = mouseValidationPath + "/" + "GSE75192"
val mouseColsValidationPath = jenageValidationPath + "/" + "expressions_columns_GSE75192.tsv"
val mouseRowsValidationPath = jenageValidationPath + "/" + "expressions_rows_GSE75192.tsv"

expressionsPath = expressions
byGoPath = expressions/go
comparison = expressions/go/gray_whale_with_bowhead_with_minke_with_NMR_with_human_with_mouse_with_cow_full_outer_counts_extended.tsv
grouped = expressions/go/grouped
ranked = expressions/go/grouped/ranked
transcriptsPath = expressions/transcripts
validationPath = expressions/validation
mouseValidationPath = expressions/validation/mouse
jenageValidationPath = expressions/validation/mouse/GSE75192
mouseColsValidationPath = expressions/validation/mouse/GSE75192/expressions_columns_GSE75192.tsv
mouseRowsValidationPath = expressions/validation/mouse/GSE75192/expressions_rows_GSE75192.tsv


expressions/validation/mouse/GSE75192/expressions_rows_GSE75192.tsv

In [ ]:
val mouseCols = readTSV(az(mouseColsValidationPath), true)

In [ ]:
%%dataframe mouseCols

In [29]:
val genes = readTSV(genesPath, true)

genes = [Name: string, ENSG00000223972.4: string ... 56201 more fields]


error: error while loading KVIterator, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-unsafe_2.11-2.3.0.jar(org/apache/spark/unsafe/KVIterator.class)' has location not matching its contents: contains class KVIterator
error: error while loading UnsafeAlignedOffset, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-unsafe_2.11-2.3.0.jar(org/apache/spark/unsafe/UnsafeAlignedOffset.class)' has location not matching its contents: contains class UnsafeAlignedOffset
error: error while loading Platform, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-unsafe_2.11-2.3.0.jar(org/apache/spark/unsafe/Platform.class)' has location not matching its contents: contains class Platform
error: error while loading UTF8String, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-unsafe_2.11-2.3.0.jar(org/apache/spark/unsafe/types/UTF8String.class)' has location not matching its contents: contains class UTF8String
error: error while loading CalendarInterval, class fi

[Name: string, ENSG00000223972.4: string ... 56201 more fields]

In [30]:
genes.count

error: error while loading Cross, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/Cross.class)' has location not matching its contents: contains class Cross
error: error while loading NaturalJoin, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/NaturalJoin.class)' has location not matching its contents: contains class NaturalJoin
error: error while loading LeftExistence, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/LeftExistence.class)' has location not matching its contents: contains class LeftExistence
error: error while loading RightOuter, class file '/usr/local/spark-2.3.0-bin-hadoop2.7/jars/spark-catalyst_2.11-2.3.0.jar(org/apache/spark/sql/catalyst/plans/RightOuter.class)' has location not matching its contents: contains class RightOuter
error: error while loading 

11689